In [1]:
import json
import imp
import os
import numpy as np
import tensorflow as tf

from model import models, data

In [17]:
imp.reload(models)
imp.reload(data)

<module 'model.data' from 'C:\\Users\\seokjong\\dev\\MyBert\\model\\data.py'>

In [70]:
with open("base_model.json") as fin:
    options = json.load(fin)

In [3]:
# https://baekyeongmin.github.io/dev/tpu-recipe-2/

def select_strategy(config) -> tf.distribute.Strategy:
    """
    Configuration을 바탕으로 Strategy를 설정합니다.
    :param config: Training / Inference Config
    :returns: tf.distribute.Strategy
    """
    if config["device"] == "GPU":
        devices: List[tf.config.PhysicalDevice] = tf.config.list_physical_devices("GPU")
        if len(devices) == 0:
            raise RuntimeError("GPU를 찾지 못했습니다. 혹시 CUDA_VISIBLE_DEVICE를 제대로 설정하셨나요?")
        if len(devices) > 1:
            strategy = tf.distribute.MirroredStrategy()
        else:
            strategy = tf.distribute.OneDeviceStrategy("/gpu:0")
    elif config["device"] == "TPU":
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=os.environ["TPU_NAME"])
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.TPUStrategy(resolver)
    else:
        raise ValueError(f"{config.device}는 지원되지 않는 기기입니다.")
    return strategy

In [4]:
config = {
    "device": "GPU"
}
strategy = select_strategy(config)
data_pattern = f"./{options['PRETRAINING_DIR']}/train*"
input_files = []
input_files.extend(tf.io.gfile.glob(data_pattern))
print(input_files)
train_dataset = data.build_interleaved_tfrecord_dataset(input_files, 128, 20, 32, 4)
train_dist_dataset  = strategy.experimental_distribute_dataset(train_dataset)

data_pattern = f"./{options['PRETRAINING_DIR']}/val*"
input_files = []
input_files.extend(tf.io.gfile.glob(data_pattern))
print(input_files)
val_dataset = data.build_interleaved_tfrecord_dataset(input_files, 128, 20, 32, 4)
val_dist_dataset  = strategy.experimental_distribute_dataset(val_dataset)

['.\\pretraining_data\\train_000.tfrecord', '.\\pretraining_data\\train_001.tfrecord', '.\\pretraining_data\\train_002.tfrecord', '.\\pretraining_data\\train_003.tfrecord', '.\\pretraining_data\\train_004.tfrecord', '.\\pretraining_data\\train_005.tfrecord', '.\\pretraining_data\\train_006.tfrecord', '.\\pretraining_data\\train_007.tfrecord', '.\\pretraining_data\\train_008.tfrecord', '.\\pretraining_data\\train_009.tfrecord', '.\\pretraining_data\\train_010.tfrecord', '.\\pretraining_data\\train_011.tfrecord', '.\\pretraining_data\\train_012.tfrecord', '.\\pretraining_data\\train_013.tfrecord', '.\\pretraining_data\\train_014.tfrecord', '.\\pretraining_data\\train_015.tfrecord', '.\\pretraining_data\\train_016.tfrecord', '.\\pretraining_data\\train_017.tfrecord', '.\\pretraining_data\\train_018.tfrecord', '.\\pretraining_data\\train_019.tfrecord', '.\\pretraining_data\\train_020.tfrecord', '.\\pretraining_data\\train_021.tfrecord', '.\\pretraining_data\\train_022.tfrecord', '.\\pretra

In [68]:
imp.reload(models)
with strategy.scope():
    model = models.BertSentimentModel(options, tf.keras.activations.gelu)
    model.compile(tf.keras.optimizers.Adam(learning_rate=1e-4, ))


In [71]:
model.load_pretrained(options)

1/1 [==============================] - 0s 14ms/step - loss: 0.6830


In [63]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_epoch9_val6.678.h5")
model.build_dummy(options, True)
model.load_weights(checkpoint_prefix, by_name=True)
# model.save_pretrained("pretrain_model", options)

1/1 [==============================] - 2s 2s/step - loss: 0.7117


In [64]:
model.summary()

Model: "bert_sentiment_model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert_main (Bert)             multiple                  4312064   
_________________________________________________________________
sentiment (SentimentAnalysis multiple                  16770     
Total params: 4,328,834
Trainable params: 4,328,834
Non-trainable params: 0
_________________________________________________________________


In [45]:
imp.reload(models)
with strategy.scope():
    model = models.BertPretrainModel(options, tf.keras.activations.gelu)
    model.compile(tf.keras.optimizers.Adam(learning_rate=1e-4, ))
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_epoch9_val6.678.h5")
model.build_dummy(options, True)
model.load_weights(checkpoint_prefix)
model.save_pretrained("pretrain_model", options)

1/1 [==============================] - 2s 2s/step - loss: 0.6431


AttributeError: 'NoneType' object has no attribute 'replace'

In [22]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_epoch{epoch}_val{val_loss:0.3f}.h5")

In [39]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_epoch9_val6.678.h5")

1/1 [==============================] - 2s 2s/step - loss: 0.7039


In [12]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True,
        monitor="val_loss"
    ),
]

model.fit(
    train_dist_dataset, 
    epochs=1000,
    callbacks=callbacks, 
    steps_per_epoch=30000, 
    validation_data=val_dist_dataset,
    validation_steps=10000
)

Epoch 1/1000
30000/30000 [==============================] - 354s 12ms/step - loss: 7.0774 - val_loss: 6.7290
Epoch 2/1000
30000/30000 [==============================] - 349s 12ms/step - loss: 6.7133 - val_loss: 6.7026
Epoch 3/1000
30000/30000 [==============================] - 350s 12ms/step - loss: 6.6916 - val_loss: 6.6896
Epoch 4/1000
30000/30000 [==============================] - 350s 12ms/step - loss: 6.6756 - val_loss: 6.6859
Epoch 5/1000
30000/30000 [==============================] - 350s 12ms/step - loss: 6.6720 - val_loss: 6.6846
Epoch 6/1000
30000/30000 [==============================] - 351s 12ms/step - loss: 6.6596 - val_loss: 6.6822
Epoch 7/1000
30000/30000 [==============================] - 345s 11ms/step - loss: 6.6546 - val_loss: 6.6816
Epoch 8/1000
30000/30000 [==============================] - 342s 11ms/step - loss: 6.6505 - val_loss: 6.6804
Epoch 9/1000
30000/30000 [==============================] - 342s 11ms/step - loss: 6.6437 - val_loss: 6.6775
Epoch 10/1000
30000

KeyboardInterrupt: 